## Singular-value decomposition

### Implementation and appplication

This notebook i) introduces the main practical aspects of computing the SVD of rectangular matrices, ii) runs examples applications and iii) shows a visual assessment of the error introduced by matrix decomposition.

#### SVD example

Today SVD is available from the `scipy` Python module, namely `scipy.linalg.svd`.

Let's see how it works on our example activity matrix. 

![](https://github.com/ale66/learn-datascience/blob/main/week-1/notation_maths/imgs/activity_matrix.png?raw=true)


$\mathbf{A} = \begin{pmatrix}
1 & 1 & 1 & 0 & 0 \\
3 & 3 & 3 & 0 & 0 \\
4 & 4 & 4 & 0 & 0 \\
5 & 5 & 5 & 0 & 0 \\
0 & 0 & 0 & 4 & 4 \\
0 & 0 & 0 & 5 & 5 \\
0 & 0 & 0 & 2 & 2
\end{pmatrix}$

#### Import modules and prepare data

In [ ]:
import numpy as np


In [ ]:
m = 7
n = 5

A = np.array((m, n), dtype=int)

A = [[1,1,1,0,0],
     [3,3,3,0,0], 
     [4,4,4,0,0], 
     [5,5,5,0,0], 
     [0,0,0,4,4], 
     [0,0,0,5,5], 
     [0,0,0,2,2]
    ]


In [ ]:
from scipy.linalg import svd

In [ ]:
# Check scipy documentation
U, singular_vals, V_t = svd(A)

##### Let's visualise the result

In [ ]:
# notice the Numpy piecewise division
U_truncated = np.trunc(U * 100) / 100

print(f'U (truncated to 2 decimals)=\n{U_truncated}')


Let's build the central matrix, which has the same shape as the original matrix. 

Some rows might be zero: the respective eigenvalue was zero.

In [ ]:
# put the singular values into the diagonal
D = np.zeros((m, n), dtype=float)

for i in range(min(m, n)):
    D[i,i] = singular_vals[i]

print(D.shape)

D_truncated = np.trunc(D * 1000) / 1000

print(f'D (truncated to 3 decimals)=\n{D_truncated}')

In [ ]:
V_t_truncated = np.trunc(V_t * 100) / 100

print(f'V^T (truncated to 2 decimals)=\n{V_t_truncated}')

#### How much numerical error was introduced?

This is a tiny matrix of integeers, so we hope that the SVD decomposition will work well.

In [ ]:
A_reconstructed = np.dot(U, np.dot(D, V_t))

A_reconstructed_trunc = np.trunc(A_reconstructed * 1000) / 1000

print(f'A_reconstructed (truncated to 3 decimals)=\n{A_reconstructed_trunc}')


### Value estimation

Thanks to the SVD, the Moore-Penrose pseudo-inverse can be computed and deployed to estimate the values of the missing entries in the matrix A. 

Remember the basic steps:

$A\vec{x} = \vec{v}$

$A^+A\vec{x} = A^+\vec{v}$

$I\vec{x} \approx A^+\vec{v}$

We compute $A^+\vec{v}$ and use it for value estimantion.

Thanks to the SVD, $A^+$ has simplified formulation:

$A^+ = VD^+U^T$

Thankfully, we have $V^T$ and $U$ from before, whereas $D^+$ is easily extracted from $D$: 

for each non-zero element $d_{ii}=\sqrt{\lambda_i}$ put its inverse $\frac{1}{\sqrt{\lambda_i}}$ in the same position of $D^+$. 

Then fill the rest with 0.

$\mathbf{D^+} = \begin{pmatrix}
\frac{1}{\sqrt{\sigma_1}} & 0 & 0 & 0 & 0 \\
0 & \frac{1}{\sqrt{\sigma_2}} & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0
\end{pmatrix}$

In [ ]:
# These assignments are not striclty necessary, but hopefully will increase readability
U_t = U.T

V = V_t.T

In [ ]:
# construct the D+ of D

D_plus = np.zeros((n, m), dtype=float)

for i in range(min(m, n)):
    
    # use condition '> 1e-6' to avoid big values.
    if D[i,i] != 0:
        D_plus[i,i] = 1 / D[i,i]


In [ ]:
# we are ready to compute the pseudo-inverse
A_plus = np.dot(V, np.dot(D_plus, U_t))


Now value estimation begins

We have a new user, Julia, who has not rated any of the items yet. 

Let's deploy SVD to estimate her ratings.

The regression/value estimation is formulated as:

$\hat{y} = a_0 + a_1 x$ 

such that the error function is minimized on the known data points:

$\sum_{i=1}^m (y_i - \hat{y}_i)^2$ 


To do so, we seek to minimise 

$A\vec{x} - v$


Thanks to SVD, the solution is simply:
    
$\vec{x} = A^+\vec{v}$

In [ ]:
x = np.dot(A_plus, A[:, -1].T)

print(x)

### Image factorization

We know look at SVD of images.

In [ ]:
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_olivetti_faces

In [ ]:
faces = fetch_olivetti_faces(shuffle=True, random_state=42)

dir(faces)

In [ ]:
olivetti_images = faces.images

image = None

In [ ]:
# define the size of the image matrix
# (64, 64) pixels
IMG_SIZE = 64

ROWS = 4

COLS = 3

In [ ]:
fig, axes = plt.subplots(nrows=ROWS, ncols=COLS, figsize=(10, 14))

for ax, img in zip(axes.flat, olivetti_images):
    image = ax.imshow(img, cmap="gray")
    fig.suptitle("A selection of Olivetti Faces", size=16)
    ax.axis("off")

if image is not None:
    fig.colorbar(image, ax=axes, orientation="horizontal", shrink=0.99, aspect=40, pad=0.01)
    
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=ROWS, ncols=COLS, figsize=(10, 14))

for ax, img in zip(axes.flat, olivetti_images):
    U, singular_vals, V_t = np.linalg.svd(img, full_matrices=False)
    
    # put the singular values into the diagonal
    D = np.zeros((IMG_SIZE, IMG_SIZE), dtype=float)

    for i in range(IMG_SIZE):
        D[i,i] = singular_vals[i]

    img_reconstructed = np.dot(U, np.dot(D, V_t))
    
    image = img_reconstructed
    image = ax.imshow(image, cmap="gray")
    fig.suptitle("SVD version of the Olivetti Faces", size=16)
    ax.axis("off")

if image is not None:
    fig.colorbar(image, ax=axes, orientation="horizontal", shrink=0.99, aspect=40, pad=0.01)
    
plt.show()